## 测试集分类

In [1]:
# 导入必要的工具包
from sklearn.model_selection import train_test_split
import numpy as np #用于数值计算
import pandas as pd #用于数据表处理，数据文件读写

In [2]:
path_train_all = '../data/new/train=all/' #训练集，特征工程后的文件 
path_train = '../data/new/train=predict/' #训练集，预测，得到的特征
train_name = '%strain=pred-%s-%s.csv' #训练集，预测特征，文件名
path_test = '../../data/new/test=class/' #测试集，预测，得到的特征
test_name = '%stest-7pay=lightgbm-%s.csv' #测试集，预测特征，文件名 

val_path = '../data/val/' #保存校验集测试结果
val_name = '%sval=stacking-base2-%s-%s.csv' #校验集预测结果文件名
summit_path = '../../data/new/test=class/' #保存提交文件，测试集测试结果 
out_name = '%stest-7pay=classBase-%s-%s.csv' # 生成的结果文件名称

train_part_num = 5 #训练集拆分个数
target = 'prediction_pay_price'
user_id = 'user_id'

## 2.读取测试集预测结果，0,1,2,3,4平均。

In [3]:
model_list = ['lightgbm']
for j, model_name in enumerate(model_list): #每个模型预测一个特征 
    print('j=%s model_name=%s' % (j, model_name))
    for i in range(train_part_num):
        if i == 0:
            col_df1 = pd.read_csv(out_name % (summit_path, model_name, i))
        else:
            col_df2 = pd.read_csv(out_name % (summit_path, model_name, i))
#             print('i=', i)
#             print(col_df1.head())
#             print(col_df2.head())
            col_df1[target] = (col_df1[target] + col_df2[target])
#             print(col_df1.head())
    col_df1[target] = (col_df1[target] / train_part_num) #求平均
    if j == 0:
        test_df = col_df1
    test_df[target] = test_df[target].astype(np.float32) #多个特征合并，得到整个数据集。
    print('test_df.shape=', test_df.shape)

j=0 model_name=lightgbm
test_df.shape= (19549, 109)


In [4]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19549 entries, 0 to 19548
Columns: 109 entries, user_id to prediction_pay_price
dtypes: float32(1), float64(2), int64(106)
memory usage: 16.2 MB


In [5]:
# 预测特征 v1 特征个数 =9
test_df.to_csv(test_name %(path_test, 'avg'), index=False)
test_df.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,14952,94,3838322,2859288,1041000,0,504000,0,6898146,4297996,...,4,4,4,4,4,3,27.166668,0.99,1,0.869829
1,15029,94,2353190,1286349,1000000,0,609000,0,3216753,513123,...,2,0,0,5,5,5,19.500000,0.99,1,0.764428
2,15059,94,6058137,3960541,5040000,2520000,2109000,0,14586512,7910807,...,0,0,0,11,11,9,13.833333,5.98,2,0.966343
3,15064,94,2958155,2350347,1364518,407296,600000,0,5895581,3241016,...,2,1,0,2,2,2,13.000000,0.99,1,0.855419
4,15102,94,1080008,19659,1000000,0,500000,0,2303911,13076,...,0,0,0,1,1,1,2.166667,0.99,1,0.933103


In [6]:
def get_again_pay(df, col, threshold):
    ''' 只保留付费记录 pay_col 付费字段'''
    df_pay = df.loc[df[col] > threshold]
    df_pay = df_pay.drop([col], axis = 1)
    df_nopay = df.loc[df[col] <= threshold]
    df_nopay = df_nopay.drop([col], axis = 1)
    return df_pay, df_nopay

In [23]:
# threshold = 0.50
# threshold = 0.89
threshold = 0.88
test_pay, test_nopay = get_again_pay(test_df, target, threshold)
print(test_df.shape)
print(test_pay.shape)
print(test_nopay.shape)
print(test_nopay.shape[0]+test_pay.shape[0])

(19549, 109)
(5343, 108)
(14206, 108)
19549


In [24]:
# 只保留付费记录
test_pay.to_csv(path_test + 'test-7pay-45pay=%s.csv' % threshold, index=False)
# 不付费的记录 
test_nopay.to_csv(path_test + 'test-7pay-45nopay=%s.csv' % threshold, index=False)
test_nopay.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14206 entries, 0 to 19548
Columns: 108 entries, user_id to pay_count
dtypes: float64(2), int64(106)
memory usage: 11.8 MB
